## Under or Over Ranked Teams

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.graph_objects as go

%matplotlib inline

In [3]:
records = pd.read_csv('scraped_results_df.csv')
# 'game_loc' indicated where the game was played at. Since the winner is listed first in 
# the original data, the '@' indicates the game was played at the loser's home.  Therefore, 
# we can create a new column called 'Winner_home' if the '@' sign is not present.
records['winner_home'] = records['game_loc']!='@'

# The rank is included in the winner and losers name within parenthesis.  The below regex will identify 
# numerical digits within the parenthesis and extract them to a new column as 'floats'.  We'll also 
# remove the rank in parenthesis from the original winner column.  We'll do this for winners and losers.
records['winner_rank'] = records['winner'].str.extract('\(([0-9]+)\)', expand=True).astype('float')
records['winner_name'] = records['winner'].str.replace('\(([0-9]+)\)', '').str.replace('\xa0', '')
records['loser_rank'] = records['loser'].str.extract('\(([0-9]+)\)', expand=True).astype('float')
records['loser_name'] = records['loser'].str.replace('\(([0-9]+)\)', '').str.replace('\xa0', '')

# Calculate a rank_diff socre.  The more negative this is, the more of an upset it is.
records['rank_diff'] = records['loser_rank'] - records['winner_rank']

# Add a pts_diff between the two pts as we can use margin of victory to see how close a 
# game is.
records['pts_diff'] = records['winner_pts'] - records['loser_pts']

# We no longer need several of these columns, so lets drop them.  
records.drop(['Unnamed: 0', 'winner','loser'], axis=1, inplace=True)

records.set_index(['year','week_number', 'row'], inplace=True)

In [4]:
records_recent = records.loc[2000:2019]



In [5]:
records_recent.groupby('loser_name').mean()[(records_recent.groupby('loser_name').count()['rank_diff'] > 5)].sort_values('rank_diff', ascending=True).head()

records_recent.groupby('winner_name').mean()[(records_recent.groupby('winner_name').count()['rank_diff'] > 5)].sort_values('rank_diff', ascending=False).head()

records_recent[(records_recent['loser_name'] == 'Iowa State') & (records_recent['rank_diff'] > 0)]

records_recent[(records_recent['loser_name'] == 'Alabama') & (records_recent['rank_diff'] > 0)]

records_recent[(records_recent['loser_name'] == 'Texas') & (records_recent['rank_diff'] > 0)]



winner_pts  loser_pts     game_date game_time game_day  \
year week_number row                                                           
2000 7           321        63.0       14.0   Oct 7, 2000       NaN      Sat   
     17          690        35.0       30.0  Dec 29, 2000       NaN      Fri   
2001 6           276        14.0        3.0   Oct 6, 2001       NaN      Sat   
2002 8           381        35.0       24.0  Oct 12, 2002       NaN      Sat   
2003 8           384        65.0       13.0  Oct 11, 2003       NaN      Sat   
2004 7           325        12.0        0.0   Oct 9, 2004       NaN      Sat   
2006 2           116        24.0        7.0   Sep 9, 2006       NaN      Sat   
2007 6           354        28.0       21.0   Oct 6, 2007       NaN      Sat   
2009 20          835        37.0       21.0   Jan 7, 2010       NaN      Thu   
2010 5           301        28.0       20.0   Oct 2, 2010       NaN      Sat   
2011 6           357        55.0       17.0   Oct 8, 2011       NaN      Sat   
                 358        55.0       17.0   Oct 8, 2011       NaN      Sat   
     7           410        38.0       26.0  Oct 15, 2011       NaN      Sat   
2012 6           383        48.0       45.0   Oct 6, 2012       NaN      Sat   
     7           423        63.0       21.0  Oct 13, 2012       NaN      Sat   
     14          796        42.0       24.0   Dec 1, 2012       NaN      Sat   
2013 12          670        38.0       13.0  Nov 16, 2013   3:30 PM      Sat   
     15          803        30.0       10.0   Dec 7, 2013   3:30 PM      Sat   
2018 11          634        42.0       41.0   Nov 3, 2018   3:30 PM      Sat   
     15          841        39.0       27.0   Dec 1, 2018  12:00 PM      Sat   

                     game_loc  \
year week_number row            
2000 7           321      NaN   
     17          690      NaN   
2001 6           276      NaN   
2002 8           381      NaN   
2003 8           384      NaN   
2004 7           325      NaN   
2006 2           116        @   
2007 6           354      NaN   
2009 20          835      NaN   
2010 5           301      NaN   
2011 6           357      NaN   
                 358      NaN   
     7           410        @   
2012 6           383        @   
     7           423      NaN   
     14          796      NaN   
2013 12          670        @   
     15          803      NaN   
2018 11          634        @   
     15          841      NaN   

                                                                  notes  \
year week_number row                                                      
2000 7           321                                                NaN   
     17          690                       Holiday Bowl (San Diego, CA)   
2001 6           276                                                NaN   
2002 8           381                                                NaN   
2003 8           384                                                NaN   
2004 7           325                                                NaN   
2006 2           116                                                NaN   
2007 6           354                                                NaN   
2009 20          835                    BCS Championship (Pasadena, CA)   
2010 5           301                                                NaN   
2011 6           357                        Cotton Bowl - Dallas, Texas   
                 358                                                NaN   
     7           410                                                NaN   
2012 6           383                                                NaN   
     7           423                                                NaN   
     14          796                                                NaN   
2013 12          670                                                NaN   
     15          803                                                NaN   
2018 11          634  Darrell K Ro